### Set up your environment

In [ ]:
ENV["OS_AUTH_URL"]="https://keystone-yeg.cloud.cybera.ca:5000/v2.0"
ENV["OS_TENANT_NAME"]="julia_workshop"
ENV["OS_PROJECT_NAME"]="julia_workshop"
ENV["OS_USERNAME"]=""
ENV["OS_PASSWORD"]=""

include(joinpath("..", "src", "lib", "Config.jl"))

### Exploring existing datasets

This framework is integrated with Swift, so that you can fetch datasets from a shared container. You can also push new datasets or transformations back to the container for your teammates to share.

Let's see what's there.

In [ ]:
Dataset.list()

We'll grab the `:titanic` dataset to start exploring.

In [ ]:
titanic = Dataset.fetch(:titanic)

Notice that the first time you run this, it will say "titanic: Found uploaded raw version". This means that it did not find the dataset locally, but did find it in the project's Swift container. It will cache this dataset locally in a *.csv* file. If you run the above code again, it will say "titanic: Found raw version", meaning it's found the dataset on the local filesystem and there's no need to download it.

### Loading modules

We're going to grab the `FreqTables` module and our own `Titanic` module. To do that, we use the `using` keyword. This will put any exported functions into the current namespace. If you want to keep from polluting the namespace and don't mind prepending the module name to function calls, you can use `import` instead.

In [8]:
using FreqTables
using Titanic

### Beginning exploration

You can use a frequency table to reduce the data down to a simple set of counts of different value types in a column.

Use a frequency table to show how many people survived vs how many died.

In [ ]:
freqtable(titanic[:Survived])

The result is a bit cryptic. There's a count of *0* and *1*, the two values that appear in the *Survived* column. The Titanic dataset uses *0* to represent "Dead" and a *1* to represent "Survived". We'll make this a little more readable soon.

Frequency tables can be used to show frequencies based on more than one column. We can use the following code to see how many men vs how many women survived.

In [ ]:
freqtable(titanic, :Sex, :Survived)

Let's deal with that *Survived* column. In *R*, you would use *factors* and *levels* to massage the data. *Julia* doesn't have any special functionality for this sort of thing, but you can get similar results by using *enums*.

Note: The "@" prefix indicates this is a Julia macro. We won't be getting too much into these. They give Julia a lot of meta-programming power. While they're often used along with regular function calls, they're a bit more complicated to create and reason about.

In [14]:
@enum SurvivedType Dead=0 Survived=1
titanic[:Survived] = to_enum(SurvivedType, titanic[:Survived])
pool!(titanic, [:Sex, :Survived])

We created the `to_enum` function as part of the framework to make converting a column to a defined *enum* easier. You can check that code out in *src/lib/Titanic.jl* if you're interested. The `pool!` function is part of the *DataFrames* module. While not strictly necessary, it can save storage space by making references to repeated data instead of copies.

The `levels` function shows us the unique values that occur in a column. We can use that to see if our conversion worked.

In [ ]:
levels(titanic[:Survived])

If we output our frequency table again for the *Sex* and *Survived* columns, we'll now see the more readable headings for *Survived*.

In [ ]:
freqtable(titanic, :Sex, :Survived)

### Visualization

We'll use the *StatPlots* module, which is part of the *JuliaPlots* ecosystem to do some quick visualizations of the data to help with our exploration. There are many different plotting libraries out there. Some are better than others in particular areas. We'll be showing plot.ly later for more interactive charts that you might want to use to present to your final audience, but during the exploration phase, you generally want to use tools that will produce quick visualizations.

In [ ]:
using StatPlots
pyplot()

We can now look at some of the same information we were using frequency tables for with pie charts.

Let's look at the proportions of female and male passengers.

In [ ]:
pie(["Female", "Male"], freqtable(titanic, :Sex))

Or how many people survived...

In [ ]:
pie(["Dead", "Survived"],freqtable(titanic, :Survived))

To visualize the survival of female vs male passengers, first divide the dataset into two groups, one with all male passengers and one with all female passengers.

Below are two different ways to accomplish the same task. The first just uses Julia and the DataFrames module's basic indexing functionality. The second is the `subset` method, which we added to the framework to allow an easier to read, more general way of subsetting.

In [ ]:
# basic Julia code
male = titanic[titanic[:Sex] .== "male",:]
female = titanic[titanic[:Sex] .== "female",:]

# via the framework:
male = subset(titanic, :Sex, "male")
female = subset(titanic, :Sex, "female")

Now we can lay out some pie charts side by side to show the survival rates of the two subsets.

In [ ]:
layout = grid(1,2)
pie(["Dead","Survived"],freqtable(male, :Survived),title="Survival Portion of Men",layout=layout)
pie!(["Dead","Survived"],freqtable(female, :Survived),title="Survival Portion of Women",layout=layout,subplot=2)

#### Statistical plots

Next up, we'll look at some more traditional statistical plots, useful for understanding the shape of your data.

The plots we're using don't deal well with *NA* values, so we're going to have to do a little data cleaning. There are many ways you might want to deal with missing values, but for now, we're just going to throw out the entire row when any column we're going to be using in our plots has an *NA* value.

In [ ]:
titanic_clean = titanic
titanic_clean = titanic_clean[~isna(titanic_clean[:Age]),:]
titanic_clean = titanic_clean[~isna(titanic_clean[:Sex]),:]
titanic_clean = titanic_clean[~isna(titanic_clean[:Survived]),:]

Boxplots help us visualize the dispersion of a set of values. The boxes represent the interquartile range (middle 50%) of the values. The whiskers extend that range 1.5x from the 1st and 3rd quartiles (the beginning and end of the middle 50%). And values beyond those points will show up as dots. This is useful for spotting data with outliers or where the mean may not be the best measure of central tendency.

In [ ]:
boxplot(titanic_clean, :Sex, :Age, title="Age Distribution by Gender", notch=true)

Above, we see that the age distribution of males and females is fairly similar, though females on board tend to be a bit younger than males overall, but there are a few outliers in male ages above the 3rd quartile + 1.5x the interquartile range.

Let's look at the age distrubution in terms of survival.

In [ ]:
boxplot(titanic_clean, :Survived, :Age, title="Age Distribution By Survival", notch=true, ylabel="Age")

In this case, both of our groups are showing outliers on the top end of the distribution. Those who survive tend to be a bit younger than those who don't.

Let's look closer at the distribution of ages. A histogram allows us to divide the values into buckets (or bins) to see the overall shape of the set. This can be useful in determining whether or not your values are normally distributed or follow some other common distribution. Depending on how the data is distributed, certain statistical tools will be more appropriate than others.

In [ ]:
histogram(titanic_clean[:Age], xlabel="Distribution of Age", ylabel="Frequency of Bucket", 
          title="Distribution of Passenger Ages on Titanic",bins=12)

The number of very young members throws the distribution off a bit, but the rest follows a normal distribution fairly closely.

To get a more precise picture, we might want to plot the density curves of these value sets. We can also divide them into separate groups based on another column.

First, let's plot the age density curves for those who survived and died.

In [ ]:
density(titanic_clean, :Age, groups=:Survived, linecolor=:auto, linewidth=3)

Interesting! Notice that the bump we noticed in the histogram above is only there for the surviving density curve. The density curve for those who died is a much more normal looking distribution. If you were really young, you probably survived, likely by getting first priority on the lifeboats.

Notice how the distribution for those how died peaks a little to the left of those who survived. So, if you were slightly younger than the average age of passengers, you were a little more likely to have bad luck on the Titanic.

Let's look at the density curves of ages divided into the female and male populations. This is the same division of data we used in our first boxplot above, but it helps us notice some new things. First, it looks like, while there were more male passengers overall, there were more female children than male children. There were more males than females at the average age, and as age increased, the male/female numbers tended to be fairly similar, right up to the oldest, where we tend to see more males again. We noticed that last one via the outliers on the boxplot.

In [ ]:
density(titanic_clean, :Age, groups=:Sex, linecolor=:auto, linewidth=3)

Let's create a brand new column to indicate whether a passenger is a child or adult, based on where we see that bump. This is called "feature engineering", and we'll be doing more of it when we dive into using some machine learning algorithms. It looks like that bump occurs around 13 years old, so we'll make that the cutoff for whether or not we classify the passenger as a child.

We'll use *Julia's* `map` function along with our `to_enum` function to accomplish this. Like many functional languages, Julia allows you to pass functions as parameters to other functions, but it also adds some syntactical sugar in special cases. When the *first* parameter of a function is another function, you can call the former by using a `do..end` block and omitting the first parameter. That `do..end` block is actually an anonymous function, but using it this way can make code easier to read. We do this with the `map` function below.

In [ ]:
@enum ChildType Child=0 Adult=1

titanic[:Child] = to_enum(ChildType, map(titanic[:Age]) do x
  if isna(x)
    NA
  elseif x < 13
    Child
  else
    Adult
  end
end)

We could have accomplished the same thing by defining our classification function separately and passing it into the `map` function, like this:

```
function classify_by_age(x)
  if isna(x)
    NA
  elseif x < 13
    Child
  else
    Adult
  end
end

titanic[:Child] = to_enum(ChildType, map(classify_by_age, titanic[:Age]))
```

We're going to do one last plot, but for that, we're also going to need to remove `NA` values like we did above. In fact, we might want to do that a lot in the future. Let's create a more general function to help us. Its parameters will be the original *DataFrame* `titanic_df`, and one or more column names to clean of `NA` values. How do you represent "one or more" in your function declaration? Julia allows us to do that by adding `...` to our last parameter. Inside the function, we'll use `colnames` without the `...` as we would any array of values.

In [ ]:
function remove_na(titanic_df, colnames...)
  ret = titanic_df;
  for colname in colnames
    ret = ret[~isna(ret[colname]),:];
  end
  ret
end

And now we can recreate our `titanic_clean` *DataFrame* with a single line of code.

In [ ]:
titanic_clean = remove_na(titanic, :Age, :Sex, :Survived, :Child)

Finally, let's make a grid of boxplots to represent the various subsets of our data. This exists in *R's* `ggplot` library, using a `facet_grid` function while constructing the plot. We couldn't find anything like it in *Julia's* various plotting libraries, but we know it can do boxplots, and we know it can do grids of plots, and that's all *R's* facet grid of boxplots is really doing. The JuliaPlots ecosystem provides some macros and recipes for creating more complex plots, and we used that. If you're interested, you can find it in *src/lib/CustomPlots.jl*, but for now, let's just see what we get... 

In [ ]:
import CustomPlots

CustomPlots.facetgridbox(titanic_clean, :Fare, xsplit=:Survived, ysplit=:Sex, boxsplit=:Child)